In [1]:
import pandas as pd
from pyspark.sql import types
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/03/05 20:03:27 WARN Utils: Your hostname, gary-demo resolves to a loopback address: 127.0.1.1; using 192.168.53.137 instead (on interface ens33)
22/03/05 20:03:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/05 20:03:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
!ls

202004-divvy-tripdata.csv		raw_taxi+_zone_lookup.parquet
202004-divvy-tripdata.zip		run_docker_db
code_1.63.2-1639562499_amd64.deb	Untitled.ipynb
dazzling-pier-338723-010edd3b54d2.json


In [7]:
df = pd.read_csv('202004-divvy-tripdata.csv', nrows = 100)

In [12]:
raw_schema = spark.createDataFrame(df).schema

In [16]:
import re
 
import re
def make_schema(raw):
    raw = str(raw)
    fields = []
   
    for i in zip([i.start() for i in re.finditer("\(",raw)][2:],[i.start() for i in re.finditer("\)",raw)][:-2]):
        fields.append(raw[i[0]+1:i[1]])
 
    schema = "types.StructType[\n"
    for f in fields:
        n = f.split(",")
        #If a column name has datetime in it. Make it a time stamp
        if '_at' in n[0]:
            n[1] = 'TimestampType'
        schema = schema+"\ttypes.StructField(\""+n[0]+"\", types."+n[1]+"(), "+n[2].capitalize()+"),\n"
    schema = schema +"])"
    print(schema)

In [17]:
make_schema(raw_schema)

types.StructType[
	types.StructField("ride_id", types.StringType(), True),
	types.StructField("rideable_type", types.StringType(), True),
	types.StructField("started_at", types.TimestampType(), True),
	types.StructField("ended_at", types.TimestampType(), True),
	types.StructField("start_station_name", types.StringType(), True),
	types.StructField("start_station_id", types.LongType(), True),
	types.StructField("end_station_name", types.StringType(), True),
	types.StructField("end_station_id", types.LongType(), True),
	types.StructField("start_lat", types.DoubleType(), True),
	types.StructField("start_lng", types.DoubleType(), True),
	types.StructField("end_lat", types.DoubleType(), True),
	types.StructField("end_lng", types.DoubleType(), True),
	types.StructField("member_casual", types.StringType(), True),
])


In [19]:
divvy_schema = types.StructType([
	types.StructField("ride_id", types.StringType(), True),
	types.StructField("rideable_type", types.StringType(), True),
	types.StructField("started_at", types.TimestampType(), True),
	types.StructField("ended_at", types.TimestampType(), True),
	types.StructField("start_station_name", types.StringType(), True),
	types.StructField("start_station_id", types.LongType(), True),
	types.StructField("end_station_name", types.StringType(), True),
	types.StructField("end_station_id", types.LongType(), True),
	types.StructField("start_lat", types.DoubleType(), True),
	types.StructField("start_lng", types.DoubleType(), True),
	types.StructField("end_lat", types.DoubleType(), True),
	types.StructField("end_lng", types.DoubleType(), True),
	types.StructField("member_casual", types.StringType(), True)
])

In [20]:
df_divvy = spark.read \
    .option("header", "true") \
    .schema(divvy_schema) \
    .csv('202004-divvy-tripdata.csv')

In [22]:
df_divvy \
    .repartition(4) \
    .write.parquet('divvy/parquet/')